In [2]:
!sudo apt-get install unzip -q

'sudo' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
import pickle
import json
import requests
import os
from PIL import Image
import pickle
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from transformers import AutoProcessor, AutoTokenizer
import random

import torch
from torch.utils.data import Dataset
from torch.utils.data import random_split, DataLoader
from transformers import AutoProcessor, AutoTokenizer


c:\Users\DDB1COB\AppData\Local\anaconda3\envs\tsai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab_Notebooks/Session30

Mounted at /content/drive
/content/drive/MyDrive/Colab_Notebooks/Session30


Step 1 - FETCH INSTRUCT150K DATA

In [12]:
url = "https://huggingface.co/datasets/liuhaotian/LLaVA-Instruct-150K/resolve/main/llava_instruct_150k.json"
response = requests.get(url)

with open("llava_instruct_150k.json", "wb") as f:
    f.write(response.content)

In [ ]:
# Opening JSON file - instruct150k
f = open('llava_instruct_150k.json')

# returns JSON object as
# a dictionary
data = json.load(f)
data[10]

{'id': '000000230238',
 'image': '000000230238.jpg',
 'conversations': [{'from': 'human',
   'value': '<image>\nWhat is the main color of the vase in the image?'},
  {'from': 'gpt', 'value': 'The main color of the vase in the image is blue.'},
  {'from': 'human', 'value': 'Where is the vase located?'},
  {'from': 'gpt',
   'value': 'The vase is located in a museum, placed on a pedestal on display.'},
  {'from': 'human', 'value': 'What kind of design is on the vase?'},
  {'from': 'gpt',
   'value': 'The vase features blue flowery drawings, which add to its decorative and artistic appeal.'},
  {'from': 'human',
   'value': 'Is there a person visible in the background of the image?'},
  {'from': 'gpt',
   'value': 'Yes, there is a person visible in the background of the image near the large vase.'},
  {'from': 'human',
   'value': 'How is the vase displayed in terms of lighting and background?'},
  {'from': 'gpt',
   'value': 'The vase is displayed with a dark background, which helps high

Step 2 - PREPARE TRAINING DATA

In [7]:


# create input pickle file by flattening the data
data_instruct150_flatten = []
r = 0

for a_idx,d in enumerate(data):
    image = d['image']
    image_url = 'http://images.cocodataset.org/train2017/' + image
    conv_iter = iter( d['conversations'])
    for i in conv_iter:
      gpt_ans = next(conv_iter)
      if len(gpt_ans['value']) > 200: # filter out too long answers
          continue
      if i['from'] == 'human' and gpt_ans['from'] == 'gpt':
        data_instruct150_flatten.append((image_url, i['value'].replace('<image>\n','').replace('\n<image>',''),gpt_ans['value']))

    if a_idx % 10000 == 0:
      print(f"{10000 * r} processed")
      r += 1



0 processed
10000 processed
20000 processed
30000 processed
40000 processed
50000 processed
60000 processed
70000 processed
80000 processed
90000 processed
100000 processed
110000 processed
120000 processed
130000 processed
140000 processed
150000 processed


In [8]:
data_instruct150_flatten[10]

('http://images.cocodataset.org/train2017/000000334872.jpg',
 'What kind of environment are they skiing in?',
 'They are skiing in a wooded environment, following a trail through the trees while surrounded by snow.')

In [ ]:
# add tokens
phi_model_name  = "microsoft/Phi-3-mini-128k-instruct"
tokenizer  = AutoTokenizer.from_pretrained(phi_model_name, trust_remote_code=True)
tokenizer.add_tokens('[QA]')
tokenizer.add_special_tokens({'pad_token':'[PAD]'})
tokenizer.pad_token, tokenizer.eos_token

NameError: name 'AutoTokenizer' is not defined

In [ ]:
# gpt like training dataset
with open('train_token.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows([['img_url','input','label']])

train_data_temp = []
r = 1
for df in data_instruct150_flatten:
  image_url = df[0]
  image_q   = df[1] + ' [QA]'
  image_a   = df[2] +  tokenizer.eos_token
  #print(image_q)
  #print(image_a)

  # tokenise
  ques_token = tokenizer(image_q, return_tensors="pt", return_attention_mask=False)['input_ids'].squeeze(0)
  ans_token  = tokenizer(image_a, return_tensors="pt", return_attention_mask=False)['input_ids'].squeeze(0)

  #print(ques_token)
  #print(ans_token)
  #break

  context_length = len(ques_token)
  combo_q_a = torch.cat( [ques_token,ans_token])

  for al in range(len(ans_token)):
    input = combo_q_a[al : al + context_length].numpy()
    label = combo_q_a[al + 1 : al + context_length + 1].numpy()
    train_data_temp.append([image_url,input,label])
    if len(train_data_temp) >= 100000: # write to the file
       print(f"Writing to disk after {r * 100000} rows")
       r += 1
       with open('train_token.csv', 'a', newline='') as file:
          writer = csv.writer(file)
          writer.writerows(train_data_temp)
       train_data_temp = []

Writing to disk after 100000 rows
Writing to disk after 200000 rows
Writing to disk after 300000 rows
Writing to disk after 400000 rows
Writing to disk after 500000 rows
Writing to disk after 600000 rows
Writing to disk after 700000 rows
Writing to disk after 800000 rows
Writing to disk after 900000 rows
Writing to disk after 1000000 rows
Writing to disk after 1100000 rows
Writing to disk after 1200000 rows
Writing to disk after 1300000 rows
Writing to disk after 1400000 rows
Writing to disk after 1500000 rows
Writing to disk after 1600000 rows
Writing to disk after 1700000 rows
Writing to disk after 1800000 rows
Writing to disk after 1900000 rows
Writing to disk after 2000000 rows
Writing to disk after 2100000 rows
Writing to disk after 2200000 rows
Writing to disk after 2300000 rows
Writing to disk after 2400000 rows
Writing to disk after 2500000 rows
Writing to disk after 2600000 rows
Writing to disk after 2700000 rows
Writing to disk after 2800000 rows
Writing to disk after 2900000

Step 3 - PYTORCH DATASET AND DATALOADER TESTING

In [ ]:
df_data = pd.read_csv('train_token.csv')
df_data.head(15)


,img_url,input,label
0,http://images.cocodataset.org/train2017/000000...,[ 1724 526 278 11955 310 278 3593 2...,[ 526 278 11955 310 278 3593 297 2...
1,http://images.cocodataset.org/train2017/000000...,[ 526 278 11955 310 278 3593 297 2...,[ 278 11955 310 278 3593 297 278 19...
2,http://images.cocodataset.org/train2017/000000...,[ 278 11955 310 278 3593 297 278 19...,[11955 310 278 3593 297 278 1967 299...
3,http://images.cocodataset.org/train2017/000000...,[11955 310 278 3593 297 278 1967 299...,[ 310 278 3593 297 278 1967 29973 320...
4,http://images.cocodataset.org/train2017/000000...,[ 310 278 3593 297 278 1967 29973 320...,[ 278 3593 297 278 1967 29973 32011 4...
5,http://images.cocodataset.org/train2017/000000...,[ 278 3593 297 278 1967 29973 32011 4...,[ 3593 297 278 1967 29973 32011 450 35...
6,http://images.cocodataset.org/train2017/000000...,[ 3593 297 278 1967 29973 32011 450 35...,[ 297 278 1967 29973 32011 450 3593 2...
7,http://images.cocodataset.org/train2017/000000...,[ 297 278 1967 29973 32011 450 3593 2...,[ 278 1967 29973 32011 450 3593 297 2...
8,http://images.cocodataset.org/train2017/000000...,[ 278 1967 29973 32011 450 3593 297 2...,[ 1967 29973 32011 450 3593 297 278 19...
9,http://images.cocodataset.org/train2017/000000...,[ 1967 29973 32011 450 3593 297 278 19...,[29973 32011 450 3593 297 278 1967 3...


In [ ]:
df_data[0:1]

,img_url,input,label
0,http://images.cocodataset.org/train2017/000000...,[ 1724 526 278 11955 310 278 3593 2...,[ 526 278 11955 310 278 3593 297 2...


In [ ]:
for i in (df_data[0:1]['input']):
    print(i)
for i in (df_data[0:1]['label']):
    print(i)

[ 1724   526   278 11955   310   278  3593   297   278  1967 29973 32011]
[  526   278 11955   310   278  3593   297   278  1967 29973 32011   450]


In [ ]:
class llavadataset(Dataset):
  def __init__(self, qa_dataset, phi_model_name, clip_model_name, tokenizer):
    self.processor  = AutoProcessor.from_pretrained(clip_model_name)
    self.qa_dataset = qa_dataset

  def __len__(self):
    return self.qa_dataset.shape[0]

  def __getitem__(self, idx):
    # from image perspective
    img_url = self.qa_dataset.img_url[idx]
    ques    = torch.tensor(np.array(np.matrix(self.qa_dataset.input[idx]))[0])
    ans     = torch.tensor(np.array(np.matrix(self.qa_dataset.label[idx]))[0])

    # image load
    image_load = Image.open(requests.get(img_url,stream=True).raw)
    image_processed = self.processor(images=image_load, return_tensors="pt") ['pixel_values']
    image_processed = image_processed.squeeze(0)
    # q = self.tokenizer(ques, return_tensors="pt", return_attention_mask=False)['input_ids'].squeeze(0)
    # a = self.tokenizer(ans, return_tensors="pt", return_attention_mask=False)['input_ids'].squeeze(0)
    return(image_processed , ques, ans)

In [ ]:
clip_model_name = "openai/clip-vit-base-patch32"
csv_file = 'train_token.csv'
qa_dataset = pd.read_csv(csv_file)
llava_dataset = llavadataset(qa_dataset, phi_model_name, clip_model_name, tokenizer)

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
llava_dataset[0]

(tensor([[[ 1.1858,  1.1566,  1.1274,  ...,  1.3902,  1.3756,  1.1712],
          [ 1.2588,  1.2296,  1.1858,  ...,  1.4048,  1.2150,  1.2734],
          [ 1.4194,  1.3318,  1.2880,  ...,  1.3610,  1.1128,  1.3756],
          ...,
          [-0.4346, -0.3762, -0.3762,  ...,  0.8209,  0.7041,  0.9522],
          [-0.4200, -0.4054, -0.3908,  ...,  0.1493,  0.1639,  0.1055],
          [-0.4346, -0.4346, -0.3908,  ...,  0.2223,  0.1493,  0.1347]],
 
         [[ 1.4145,  1.3695,  1.3395,  ...,  1.6397,  1.6247,  1.3995],
          [ 1.4596,  1.4446,  1.3995,  ...,  1.6397,  1.4446,  1.5046],
          [ 1.5646,  1.4896,  1.4295,  ...,  1.5946,  1.3395,  1.6096],
          ...,
          [-0.4014, -0.3264, -0.3264,  ...,  0.9793,  0.7542,  1.0393],
          [-0.3564, -0.3414, -0.3264,  ...,  0.2439,  0.2289,  0.1839],
          [-0.3714, -0.3714, -0.3264,  ...,  0.2439,  0.1989,  0.1839]],
 
         [[ 1.7904,  1.7904,  1.7477,  ...,  1.9895,  1.9610,  1.7477],
          [ 1.8473,  1.8331,

In [ ]:
def collate_fn(batch):
    image_embeddings, ques, ans = zip(*batch)
    image_embeddings_stacked = torch.stack(image_embeddings, dim=0)
    ques_padded = torch.nn.utils.rnn.pad_sequence(ques, batch_first=True, padding_value=tokenizer.pad_token_id)
    ans_padded = torch.nn.utils.rnn.pad_sequence(ans, batch_first=True, padding_value=tokenizer.pad_token_id)
    return (image_embeddings_stacked, ques_padded,ans_padded)


In [ ]:
tokenizer.pad_token_id,tokenizer.eos_token_id

(32012, 32000)

In [ ]:
val_dataloader   = DataLoader(llavadataset(qa_dataset, phi_model_name,clip_model_name,tokenizer),
                      collate_fn=collate_fn, batch_size=2, num_workers = 10, shuffle=True, pin_memory=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
next(iter(val_dataloader))

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


(tensor([[[[-1.7631, -1.7485, -1.7485,  ..., -1.7923, -1.7923, -1.7923],
           [-1.7777, -1.7193, -1.7485,  ..., -1.7923, -1.7923, -1.7923],
           [-1.7631, -1.7193, -1.7485,  ..., -1.7923, -1.7923, -1.7923],
           ...,
           [ 0.8355,  0.8063,  0.8501,  ..., -1.1791, -1.2083, -1.2375],
           [ 0.7187,  0.7917,  0.8209,  ..., -0.4054, -0.3908, -0.3470],
           [ 0.6457,  0.7041,  0.7479,  ..., -0.1426, -0.0696, -0.1572]],
 
          [[-1.7221, -1.7221, -1.7071,  ..., -1.7521, -1.7521, -1.7521],
           [-1.7371, -1.6921, -1.7221,  ..., -1.7521, -1.7521, -1.7521],
           [-1.7371, -1.6921, -1.7221,  ..., -1.7521, -1.7521, -1.7521],
           ...,
           [ 0.8292,  0.7992,  0.8292,  ..., -1.1518, -1.2268, -1.2118],
           [ 0.7842,  0.7842,  0.7692,  ..., -0.1913, -0.1163, -0.1313],
           [ 0.7242,  0.7242,  0.7242,  ..., -0.0562, -0.0412, -0.0262]],
 
          [[-1.4660, -1.3807, -1.3665,  ..., -1.4802, -1.4802, -1.4802],
           [-


Step 4 - SAMPLE VALIDATION DATA

In [ ]:
# Opening JSON file - instruct150k
f = open('llava_instruct_150k.json')

# returns JSON object as
# a dictionary
data = json.load(f)

In [ ]:
# create input pickle file by flattening the data
data_instruct150_sample_val_flatten = []
r = 0

for a_idx,d in enumerate(data):
    image = d['image']
    image_url = 'http://images.cocodataset.org/train2017/' + image
    conv_iter = iter( d['conversations'])
    for i in conv_iter:
      gpt_ans = next(conv_iter)
      if len(gpt_ans['value']) > 200: # filter out too long answers
          continue
      if i['from'] == 'human' and gpt_ans['from'] == 'gpt':
        image_q   = i['value'].replace('<image>\n','').replace('\n<image>','') + ' [QA]'
        image_a   = gpt_ans['value'] +  tokenizer.eos_token
        data_instruct150_sample_val_flatten.append([image_url, image_q, image_a ])

    if a_idx % 10000 == 0:
      print(f"{10000 * r} processed")
      r += 1
      if r >= 2:
        break


0 processed
10000 processed


In [ ]:
data_instruct150_sample_val_flatten[123]


['http://images.cocodataset.org/train2017/000000432526.jpg',
 'What are the skiers doing on the slope? [QA]',
 'The skiers have stopped on the side of the slope and are posing for a photo.<|endoftext|>']

In [ ]:
with open('sample_val_data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows([['img_url','q','a']])

# data
with open('sample_val_data.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data_instruct150_sample_val_flatten)
